In [1]:
import requests
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from fuzzywuzzy import process

In [8]:
# Section 1: Load all data and verify

def load_pickle_data(urls):
    loaded_data = []
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            data = pickle.loads(response.content)
            loaded_data.append(data)
        else:
            print(f"Failed to retrieve {url}")
    return loaded_data

# URLs for the files to be pulled from GitHub
urls = [
    'https://github.com/mamaOcoder/highspeedrail/raw/dcb2607fc164695b682e18bebaf35923e127094f/data/pickled/distance_df.pickle',
    'https://github.com/mamaOcoder/highspeedrail/raw/dcb2607fc164695b682e18bebaf35923e127094f/data/pickled/flights_df.pickle',
    'https://github.com/mamaOcoder/highspeedrail/raw/dcb2607fc164695b682e18bebaf35923e127094f/data/pickled/flights_df_all_fields.pickle',
    'https://github.com/mamaOcoder/highspeedrail/raw/dcb2607fc164695b682e18bebaf35923e127094f/data/pickled/geos_df.pickle',
    'https://github.com/mamaOcoder/highspeedrail/raw/dcb2607fc164695b682e18bebaf35923e127094f/data/pickled/msa_df.pickle',
    'https://github.com/mamaOcoder/highspeedrail/raw/88761104ba44f569e5fadde614eb5ed6a54900ba/data/pickled/tti_df.pickle'
]

# Github Data
distance_df, flights_df, flights_df_all_fields, geos_df, msa_df, tti_df = map(pd.DataFrame, load_pickle_data(urls))

# Files that I had to pull locally
city_pair_weights_df = pd.read_pickle('city_pair_weights_df.pickle')
msa_weights_df = pd.read_pickle('msa_weights_df.pickle')
city_pair_df = pd.read_pickle('city_pair_df.pickle')

# Verification 
print(distance_df.head())
print(city_pair_weights_df.head())
print(msa_weights_df.head())
print(city_pair_df.head())


        Origin      Destination  Distance_meters  Distance_miles  \
0  Abilene, TX        Akron, OH        2137508.0     1328.185483   
1  Abilene, TX       Albany, GA        1582948.0      983.597982   
2  Abilene, TX       Albany, OR        3077199.0     1912.082220   
3  Abilene, TX       Albany, NY        2917051.0     1812.570897   
4  Abilene, TX  Albuquerque, NM         783883.0      487.082164   

   Duration_seconds  Duration_minutes  
0           69643.0            1161.0  
1           54532.0             909.0  
2          104836.0            1747.0  
3           94703.0            1578.0  
4           26964.0             449.0  
         City1            City2  drive_distance  drive_duration  \
0  Abilene, TX        Akron, OH     1327.729086          1159.0   
1  Abilene, TX       Albany, GA      982.946785           910.0   
2  Abilene, TX       Albany, NY     1812.063858          1578.5   
3  Abilene, TX       Albany, OR     1912.769767          1746.5   
4  Abilene, TX  

In [9]:
# Section 2: Data Preparation
city_column_name = 'MetroArea'

def get_best_match(city, choices):
    match, score = process.extractOne(city, choices)
    return match if score > 80 else None

# Fuzzy matching to align TTI city names with other datasets
all_city_names = set(msa_df['MainCity']).union(set(distance_df['Origin']), set(distance_df['Destination']))
tti_df['Matched_City'] = tti_df[city_column_name].apply(lambda x: get_best_match(x, all_city_names))

# Convert TTI DataFrame to a dictionary for quick lookup
tti_data = tti_df.set_index('Matched_City')['TravelTimeIndexValue'].to_dict()

# Merge msa_df with geos_df to get population and GDP for each city
msa_merged_df = pd.merge(msa_df, geos_df, left_on='MainCity', right_on='MainCity')
city_gdp = msa_merged_df.set_index('MainCity')['GDP_thousands_dollars'].to_dict()


In [48]:
# Section 3: Scoring Functions
def score_distance(dist):
    mean_distance = 300  # Optimal distance for HSR
    std_dev = 100  
    score = np.exp(-0.5 * ((dist - mean_distance) / std_dev) ** 2) * 100
    return score

def score_gdp(gdp):
    return (gdp / max(city_gdp.values())) * 100

def score_congestion(ttis, city):
    return (ttis.get(city, 0) / max(ttis.values())) * 100

def score_emissions(distance):
    hsr_emissions_factor = 0.0045
    air_travel_emissions_factor = 0.115
    hsr_emissions = distance * hsr_emissions_factor
    air_emissions = distance * air_travel_emissions_factor
    emissions_savings = air_emissions - hsr_emissions
    return emissions_savings


In [45]:
# Section 4: Merge and score with additional criteria and added weights

# Merge city pair weights and MSA weights into the city_pairs DataFrame
city_pairs = pd.merge(distance_df[['Origin', 'Destination', 'Distance_miles']], 
                      city_pair_weights_df[['City1', 'City2', 'gaus_dist_weight', 'demand_weight']],
                      left_on=['Origin', 'Destination'], 
                      right_on=['City1', 'City2'],
                      how='left')

city_pairs = pd.merge(city_pairs, 
                      msa_weights_df[['MetroArea', 'demand_score', 'proximity_score', 'cluster_id']],
                      left_on='Origin', 
                      right_on='MetroArea',
                      how='left')

city_pairs = pd.merge(city_pairs, 
                      msa_weights_df[['MetroArea', 'demand_score', 'proximity_score', 'cluster_id']],
                      left_on='Destination', 
                      right_on='MetroArea',
                      how='left',
                      suffixes=('_Origin', '_Destination'))

# Filtering for AK, HI, and distance >1000km (This needs to be done before scoring to properly normalize)
filtered_city_pairs = city_pairs[
    ~city_pairs['Origin'].str.contains(', AK|, HI') &
    ~city_pairs['Destination'].str.contains(', AK|, HI') &
    (city_pairs['Distance_miles'] <= 1000)
]

# Function to score city pairs with updated criteria
def score_city_pairs_updated(city_pairs):
    city_pairs['Distance_Score'] = city_pairs['Distance_miles'].map(score_distance)
    city_pairs['GDP_Score_Origin'] = city_pairs['Origin'].map(lambda x: score_gdp(city_gdp.get(x, 0)))
    city_pairs['GDP_Score_Destination'] = city_pairs['Destination'].map(lambda x: score_gdp(city_gdp.get(x, 0)))
    city_pairs['Congestion_Score_Origin'] = city_pairs['Origin'].map(lambda x: score_congestion(tti_data, x))
    city_pairs['Congestion_Score_Destination'] = city_pairs['Destination'].map(lambda x: score_congestion(tti_data, x))
    city_pairs['Emissions_Score'] = city_pairs['Distance_miles'].map(lambda x: score_emissions(x * 1.60934))

    # Include new weights from data mapping (Leslie)
    city_pairs['Gaussian_Score'] = city_pairs['gaus_dist_weight']
    city_pairs['Demand_Score'] = city_pairs['demand_weight'] + city_pairs['demand_score_Origin'] + city_pairs['demand_score_Destination']
    city_pairs['Proximity_Score'] = city_pairs['proximity_score_Origin'] + city_pairs['proximity_score_Destination']

    # Add cluster-based scoring (more weight towards cluster alignment in place of assigning 'megaregions')
    city_pairs['Cluster_Score'] = np.where(city_pairs['cluster_id_Origin'] == city_pairs['cluster_id_Destination'], 10, 5)
    city_pairs['Cluster_Score'] = np.where(city_pairs['Demand_Score'] > 5, 15, city_pairs['Cluster_Score'])

    # Sum the scores to get a total score for each city pair (Can either adjust weights here or change criteria scoring above)
    city_pairs['Total_Score'] = (
        0.1 * (city_pairs['Distance_Score']) +  
        0.8 * (city_pairs['GDP_Score_Origin'] + city_pairs['GDP_Score_Destination']) + 
        0.01 * (city_pairs['Congestion_Score_Origin'] + city_pairs['Congestion_Score_Destination']) + 
        0.1 * city_pairs['Emissions_Score'] + 
        100 * city_pairs['Gaussian_Score'] + 
        1.8 * city_pairs['Demand_Score'] + 
        1.5 * city_pairs['Proximity_Score'] + 
        2.0 * city_pairs['Cluster_Score']
    )

    return city_pairs

In [46]:
# Apply the updated scoring to the filtered city pairs (must come after filtering)
filtered_city_pairs = score_city_pairs_updated(filtered_city_pairs)

# Normalize
max_score = filtered_city_pairs['Total_Score'].max()
filtered_city_pairs['Normalized_Score'] = (filtered_city_pairs['Total_Score'] / max_score) * 100


/var/folders/71/kt3th06j0w36j864cy126bwm0000gn/T/ipykernel_75528/2310171062.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_pairs['Distance_Score'] = city_pairs['Distance_miles'].map(score_distance)
/var/folders/71/kt3th06j0w36j864cy126bwm0000gn/T/ipykernel_75528/2310171062.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_pairs['GDP_Score_Origin'] = city_pairs['Origin'].map(lambda x: score_gdp(city_gdp.get(x, 0)))
/var/folders/71/kt3th06j0w36j864cy126bwm0000gn/T/ipykernel_75528/2310171062

In [47]:
# Rank by their normalized scores
top_50_filtered_city_pairs = filtered_city_pairs.nlargest(50, 'Normalized_Score')

# Display the top 50 city pairs
print(top_50_filtered_city_pairs[['Origin', 'Destination', 'Distance_miles', 
                                  'Distance_Score', 'GDP_Score_Origin', 
                                  'GDP_Score_Destination', 'Congestion_Score_Origin', 'Congestion_Score_Destination', 
                                  'Emissions_Score', 'Gaussian_Score', 'Demand_Score', 
                                  'Proximity_Score', 'Cluster_Score', 'Total_Score', 'Normalized_Score']])

                    Origin        Destination  Distance_miles  Distance_Score  \
44137         Florence, SC   Jacksonville, FL      310.311435       99.469782   
1999           Abilene, TX       Longview, TX      308.721346       99.620413   
14415         Billings, MT     Rapid City, SD      316.441881       98.657417   
14931       Birmingham, AL    Spartanburg, SC      322.171543       97.572073   
88593          Midland, TX           Waco, TX      320.275740       97.965454   
34427           Dalton, GA       Valdosta, GA      317.411841       98.495570   
65304          Jackson, MS          Tyler, TX      312.463242       99.226346   
14966      Bloomington, IL    Springfield, OH      300.797002       99.996824   
48419      Fond du Lac, WI        Mankato, MN      306.227163       99.806300   
92307           Mobile, AL       Valdosta, GA      316.380365       98.667377   
63615          Jackson, TN      Knoxville, TN      307.871932       99.690643   
63776        Iowa City, IA  

## Results 
- Thus far the results show the score for each of the criteria however, they are not weighted properly as you can see by the drastically large scores for congestion, emissions, gaussian (super low), cluster (arguably isn't awful). 
- Also having some null values show up
- Distance is currently optimal around 300 miles via normal distribution around 300 miles instead of a linear scoring mechanism